In [1]:
import requests
from bs4 import BeautifulSoup
import time
import re
import random
import pandas as pd

In [2]:
Movies = pd.read_csv('Movielist_Marvel.csv')
Movies

,name,name_link
0,Captain America: The First Avenger,captain_america_the_first_avenger
1,Iron Man,iron_man
2,The Incredible Hulk,the_incredible_hulk
3,Iron Man 2,iron_man_2
4,Thor,thor
5,The Avengers,marvels_the_avengers
6,Iron Man Three,iron_man_3
7,Thor: The Dark World,thor_the_dark_world
8,Captain America: The Winter Soldier,captain_america_the_winter_soldier_2014
9,Guardians of the Galaxy,guardians_of_the_galaxy


In [3]:
user_agents = ['Mozilla/5.0 (Windows NT 6.1; rv:2.0.1) Gecko/20100101 Firefox/4.0.1',
               'Mozilla/5.0 (Windows; U; Windows NT 6.1; en-us) AppleWebKit/534.50 (KHTML, like Gecko) Version/5.1 Safari/534.50',
               'Opera/9.80 (Windows NT 6.1; U; en) Presto/2.8.131 Version/11.11']

In [4]:
def find_star_missing(soup):
    # find the indexes for star missing
    star_missing = []
    review_frame=soup.find_all('div',class_='col-xs-16')
    for i in range(len(review_frame)):
        if not review_frame[i].find('span', class_="glyphicon glyphicon-star") and not review_frame[i].find(text = re.compile('\xbd')):
            star_missing.append(i)
    return star_missing

def crawler(url):
    # fetch html by url
    headers = {'User-Agent':random.choice(user_agents)}
    r = requests.get(url, timeout=(3.05, 10), headers = headers)
    soup = BeautifulSoup(r.text, 'html.parser')

    # parse html
    try:
        data = []
        user_review=soup.find_all('div',class_='user_review')
        stars=soup.find_all('span',class_='fl')
        date=soup.find_all('span',class_='fr small subtle')
        for i in range(len(user_review)):        
            stars_single = len(stars[i].find_all('span', class_="glyphicon glyphicon-star")) + float(len(stars[i].text)-1)/2            
            data.append((user_review[i].text, stars_single, date[i].text))
            
    except: # star missing
        data = []
        user_review=soup.find_all('div',class_='user_review')
        stars=soup.find_all('span',class_='fl')
        date=soup.find_all('span',class_='fr small subtle')
        
        # insert star missing
        star_missing = find_star_missing(soup)
        for insert_index in star_missing:    
            stars.insert(insert_index,0)
            
        for i in range(len(user_review)):
            if stars[i] == 0:
                stars_single = 0
            else:
                stars_single = len(stars[i].find_all('span', class_="glyphicon glyphicon-star")) + float(len(stars[i].text)-1)/2             
            data.append((user_review[i].text, stars_single, date[i].text))
            
    return data

In [5]:
for name_link in Movies['name_link']:
    index = Movies.index[Movies['name_link'] == name_link].tolist()[0]
    data = []
    # 1st page
    url = 'https://www.rottentomatoes.com/m/' + name_link + '/reviews/?type=user&sort='
    data = crawler(url)
    print(name_link, "page%d"%1, "Total reviews:%d"%len(data))
    # 2-51st page
    for i in range(2,52):
        attempts = 0
        success = False
        while attempts < 3 and not success:    
            try:
                time.sleep(random.uniform(1, 2))
                url = 'https://www.rottentomatoes.com/m/' + name_link + '/reviews/?page='+ str(i) + '&type=user&sort='
                data_page = crawler(url)
                data = data + data_page
                print(name_link, "page%d"%i, "Total reviews:%d"%len(data_page))
                success = True
            except:
                attempts += 1
                if attempts == 3:
                    print(i, 'error')
    # output
    df = pd.DataFrame(data, columns=['review', 'stars', 'date'])
    df.to_csv('Reviews_Marvel/'+Movies['name'][index]+'.csv', index=False, encoding='utf-8')
    print(name_link, "Finished task %d"%index, "Total reviews:%d"%len(df))

('captain_america_the_first_avenger', 'page1', 'Total reviews:20')
('captain_america_the_first_avenger', 'page2', 'Total reviews:19')
('captain_america_the_first_avenger', 'page3', 'Total reviews:20')
('captain_america_the_first_avenger', 'page4', 'Total reviews:20')
('captain_america_the_first_avenger', 'page5', 'Total reviews:20')
('captain_america_the_first_avenger', 'page6', 'Total reviews:20')
('captain_america_the_first_avenger', 'page7', 'Total reviews:20')
('captain_america_the_first_avenger', 'page8', 'Total reviews:19')
('captain_america_the_first_avenger', 'page9', 'Total reviews:19')
('captain_america_the_first_avenger', 'page10', 'Total reviews:20')
('captain_america_the_first_avenger', 'page11', 'Total reviews:20')
('captain_america_the_first_avenger', 'page12', 'Total reviews:20')
('captain_america_the_first_avenger', 'page13', 'Total reviews:20')
('captain_america_the_first_avenger', 'page14', 'Total reviews:20')
('captain_america_the_first_avenger', 'page15', 'Total re

('the_incredible_hulk', 'page46', 'Total reviews:20')
('the_incredible_hulk', 'page47', 'Total reviews:20')
('the_incredible_hulk', 'page48', 'Total reviews:20')
('the_incredible_hulk', 'page49', 'Total reviews:20')
('the_incredible_hulk', 'page50', 'Total reviews:20')
('the_incredible_hulk', 'page51', 'Total reviews:20')
('the_incredible_hulk', 'Finished task 2', 'Total reviews:1010')
('iron_man_2', 'page1', 'Total reviews:19')
('iron_man_2', 'page2', 'Total reviews:20')
('iron_man_2', 'page3', 'Total reviews:20')
('iron_man_2', 'page4', 'Total reviews:20')
('iron_man_2', 'page5', 'Total reviews:20')
('iron_man_2', 'page6', 'Total reviews:20')
('iron_man_2', 'page7', 'Total reviews:18')
('iron_man_2', 'page8', 'Total reviews:20')
('iron_man_2', 'page9', 'Total reviews:20')
('iron_man_2', 'page10', 'Total reviews:20')
('iron_man_2', 'page11', 'Total reviews:19')
('iron_man_2', 'page12', 'Total reviews:20')
('iron_man_2', 'page13', 'Total reviews:20')
('iron_man_2', 'page14', 'Total rev

('iron_man_3', 'page14', 'Total reviews:20')
('iron_man_3', 'page15', 'Total reviews:19')
('iron_man_3', 'page16', 'Total reviews:20')
('iron_man_3', 'page17', 'Total reviews:19')
('iron_man_3', 'page18', 'Total reviews:20')
('iron_man_3', 'page19', 'Total reviews:20')
('iron_man_3', 'page20', 'Total reviews:20')
('iron_man_3', 'page21', 'Total reviews:20')
('iron_man_3', 'page22', 'Total reviews:20')
('iron_man_3', 'page23', 'Total reviews:20')
('iron_man_3', 'page24', 'Total reviews:20')
('iron_man_3', 'page25', 'Total reviews:19')
('iron_man_3', 'page26', 'Total reviews:19')
('iron_man_3', 'page27', 'Total reviews:20')
('iron_man_3', 'page28', 'Total reviews:19')
('iron_man_3', 'page29', 'Total reviews:19')
('iron_man_3', 'page30', 'Total reviews:20')
('iron_man_3', 'page31', 'Total reviews:20')
('iron_man_3', 'page32', 'Total reviews:20')
('iron_man_3', 'page33', 'Total reviews:20')
('iron_man_3', 'page34', 'Total reviews:20')
('iron_man_3', 'page35', 'Total reviews:20')
('iron_man

('captain_america_the_winter_soldier_2014', 'page50', 'Total reviews:20')
('captain_america_the_winter_soldier_2014', 'page51', 'Total reviews:20')
('captain_america_the_winter_soldier_2014', 'Finished task 8', 'Total reviews:1005')
('guardians_of_the_galaxy', 'page1', 'Total reviews:20')
('guardians_of_the_galaxy', 'page2', 'Total reviews:20')
('guardians_of_the_galaxy', 'page3', 'Total reviews:20')
('guardians_of_the_galaxy', 'page4', 'Total reviews:20')
('guardians_of_the_galaxy', 'page5', 'Total reviews:20')
('guardians_of_the_galaxy', 'page6', 'Total reviews:20')
('guardians_of_the_galaxy', 'page7', 'Total reviews:20')
('guardians_of_the_galaxy', 'page8', 'Total reviews:20')
('guardians_of_the_galaxy', 'page9', 'Total reviews:19')
('guardians_of_the_galaxy', 'page10', 'Total reviews:20')
('guardians_of_the_galaxy', 'page11', 'Total reviews:20')
('guardians_of_the_galaxy', 'page12', 'Total reviews:19')
('guardians_of_the_galaxy', 'page13', 'Total reviews:20')
('guardians_of_the_gal

('antman', 'page37', 'Total reviews:19')
('antman', 'page38', 'Total reviews:20')
('antman', 'page39', 'Total reviews:19')
('antman', 'page40', 'Total reviews:20')
('antman', 'page41', 'Total reviews:20')
('antman', 'page42', 'Total reviews:20')
('antman', 'page43', 'Total reviews:20')
('antman', 'page44', 'Total reviews:20')
('antman', 'page45', 'Total reviews:19')
('antman', 'page46', 'Total reviews:20')
('antman', 'page47', 'Total reviews:19')
('antman', 'page48', 'Total reviews:20')
('antman', 'page49', 'Total reviews:19')
('antman', 'page50', 'Total reviews:20')
('antman', 'page51', 'Total reviews:20')
('antman', 'Finished task 12', 'Total reviews:1004')
('captain_america_civil_war', 'page1', 'Total reviews:20')
('captain_america_civil_war', 'page2', 'Total reviews:20')
('captain_america_civil_war', 'page3', 'Total reviews:19')
('captain_america_civil_war', 'page4', 'Total reviews:20')
('captain_america_civil_war', 'page5', 'Total reviews:20')
('captain_america_civil_war', 'page6'

('doctor_strange_2016', 'page29', 'Total reviews:20')
('doctor_strange_2016', 'page30', 'Total reviews:20')
('doctor_strange_2016', 'page31', 'Total reviews:20')
('doctor_strange_2016', 'page32', 'Total reviews:20')
('doctor_strange_2016', 'page33', 'Total reviews:20')
('doctor_strange_2016', 'page34', 'Total reviews:20')
('doctor_strange_2016', 'page35', 'Total reviews:20')
('doctor_strange_2016', 'page36', 'Total reviews:20')
('doctor_strange_2016', 'page37', 'Total reviews:19')
('doctor_strange_2016', 'page38', 'Total reviews:20')
('doctor_strange_2016', 'page39', 'Total reviews:20')
('doctor_strange_2016', 'page40', 'Total reviews:20')
('doctor_strange_2016', 'page41', 'Total reviews:20')
('doctor_strange_2016', 'page42', 'Total reviews:20')
('doctor_strange_2016', 'page43', 'Total reviews:20')
('doctor_strange_2016', 'page44', 'Total reviews:20')
('doctor_strange_2016', 'page45', 'Total reviews:20')
('doctor_strange_2016', 'page46', 'Total reviews:20')
('doctor_strange_2016', 'pag

('avengers_infinity_war', 'page26', 'Total reviews:20')
('avengers_infinity_war', 'page27', 'Total reviews:19')
('avengers_infinity_war', 'page28', 'Total reviews:19')
('avengers_infinity_war', 'page29', 'Total reviews:20')
('avengers_infinity_war', 'page30', 'Total reviews:20')
('avengers_infinity_war', 'page31', 'Total reviews:19')
('avengers_infinity_war', 'page32', 'Total reviews:20')
('avengers_infinity_war', 'page33', 'Total reviews:20')
('avengers_infinity_war', 'page34', 'Total reviews:20')
('avengers_infinity_war', 'page35', 'Total reviews:19')
('avengers_infinity_war', 'page36', 'Total reviews:20')
('avengers_infinity_war', 'page37', 'Total reviews:20')
('avengers_infinity_war', 'page38', 'Total reviews:20')
('avengers_infinity_war', 'page39', 'Total reviews:20')
('avengers_infinity_war', 'page40', 'Total reviews:20')
('avengers_infinity_war', 'page41', 'Total reviews:18')
('avengers_infinity_war', 'page42', 'Total reviews:20')
('avengers_infinity_war', 'page43', 'Total revie

# Merge csv files
https://www.cnblogs.com/taolusi/p/9292995.html

In [16]:
for i in Movies['name']:
    df = pd.read_csv('Reviews_Marvel/'+i+'.csv')
    df['name'] = i
    df.to_csv('Reviews_Marvel.csv', mode='a+', index=False, header=False)
    print(df.shape[0], i)

(1006, 'Captain America: The First Avenger')
(1007, 'Iron Man')
(1010, 'The Incredible Hulk')
(1007, 'Iron Man 2')
(1009, 'Thor')
(1008, 'The Avengers')
(1005, 'Iron Man Three')
(1010, 'Thor: The Dark World')
(1005, 'Captain America: The Winter Soldier')
(1010, 'Guardians of the Galaxy')
(1012, 'Guardians of the Galaxy Vol. 2')
(1008, 'Avengers: Age of Ultron')
(1004, 'Ant-Man')
(1008, 'Captain America: Civil War')
(1017, 'Spider-Man: Homecoming')
(1015, 'Doctor Strange')
(1015, 'Thor: Ragnarok')
(1013, 'Black Panther')
(1009, 'Avengers: Infinity War')
(1013, 'Ant-Man and the Wasp')
(1014, 'Captain Marvel')


In [17]:
df = pd.read_csv('Reviews_Marvel/'+i+'.csv')
df.head()

,review,stars,date
0,"I don't care about what people say, I think ...",5.0,"April 13, 2019"
1,Captain Marvel wasn't as great as many of Ma...,3.0,"April 13, 2019"
2,"Really decent movie, rounds it together well...",4.0,"April 13, 2019"
3,"A solid, entertaining marvel film with amazi...",4.0,"April 13, 2019"
4,"Slow at first but was really good, kind of f...",3.0,"April 13, 2019"


In [18]:
# add header & column order
df = pd.read_csv('Reviews_Marvel.csv', header=None).drop_duplicates()
df.columns = ['review', 'stars', 'date', 'name']
columns = ['name', 'review', 'stars', 'date']
df.to_csv('Reviews_Marvel.csv', index=False, columns=columns)
print(df.shape)
df.head()

(21186, 4)


,review,stars,date,name
0,Captain America: The First Avenger might not...,3.5,"April 11, 2019",Captain America: The First Avenger
1,"It starts well, but later, it goes like a ca...",3.0,"April 11, 2019",Captain America: The First Avenger
2,"Chris Evans, with a physique so chiseled, ca...",2.0,"April 9, 2019",Captain America: The First Avenger
3,(78/100)Captain America: The First Avenger i...,4.0,"April 8, 2019",Captain America: The First Avenger
4,The overly patriotic aspect of the character...,1.0,"April 7, 2019",Captain America: The First Avenger


In [19]:
df = pd.read_csv('Reviews_Marvel.csv')
print(df.shape)
df.head()

(21186, 4)


,name,review,stars,date
0,Captain America: The First Avenger,Captain America: The First Avenger might not...,3.5,"April 11, 2019"
1,Captain America: The First Avenger,"It starts well, but later, it goes like a ca...",3.0,"April 11, 2019"
2,Captain America: The First Avenger,"Chris Evans, with a physique so chiseled, ca...",2.0,"April 9, 2019"
3,Captain America: The First Avenger,(78/100)Captain America: The First Avenger i...,4.0,"April 8, 2019"
4,Captain America: The First Avenger,The overly patriotic aspect of the character...,1.0,"April 7, 2019"


In [20]:
len(df['name'].unique())

21